# Run experiments to "Validate Mathis model at Edge and Core Scale"

## Set up your FABRIC environment


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [ ]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [ ]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Setup the scripts and modules

First, we will upload all the scripts we need to analyze the results for this experiment. This needs to be done only once before start of experiment.

In [ ]:
for n in sender_nodes:
    n.upload_file('process_cwn_file.py','process_cwn_file.py')
    n.upload_file('process_iperf_normal.py','process_iperf_normal.py')
    n.upload_file('mathis_sender.py','mathis_sender.py')
    n.upload_file("cwn.sh", "cwn.sh")

Also we need sklearn to process the downloaded files, so install the library in the environment.This needs to be done only once before start of experiment.

In [ ]:
pip install scikit-learn

## Generate flows

### Set experiment parameters

>delay, cca, test_duration, num_servers, flows

delay is the delay to be set at the receiver (=20 ms for this experiment)

cca is the congestion control algorithm (reno for this experiment)

Vary below parameters only:

num_servers is the number of ports to be opened on each receiver. For core scale we are opening 10 ports and for edge scale we are opening 1 port.

test_duration is the time for which to send the iperf3 flows.

flows is the number of parallel flows to be send from each port.

For example, to send 1000 flows set num_servers=10 and flows=10. This will send 100 flows from each of the 10 senders.

interval is to set the interval time in seconds between periodic bandwidth, jitter, and loss reports from iperf3 (=0.01 sec)

In [ ]:
cca="reno"
delay=20
test_duration=800
num_servers=10
flows=10
interval=0.01

### Get queue statistics on the router before experiment

In [ ]:
router_node.execute("tc -p -s -d -j qdisc show dev "+router_egress_name +" >tc_before.txt")

In [ ]:
router_node.execute("tail --lines=10 tc_before.txt")

## Remove existing result files from the hosts

In [ ]:
for n in receiver_nodes:
    n.execute("rm -f ./*")

for n in sender_nodes:
    n.execute("rm -f sender*")
    n.execute("rm -f data*")
    n.execute("rm -f packet*")
    n.execute("rm -f output*")

Check if the files are removed from the senders and receivers

In [ ]:
for n in sender_nodes:
    n.execute("ls")

In [ ]:
for n in receiver_nodes:
    n.execute("ls")

### Set delay on the receiver

> Values: 20ms, 100ms, 200ms

Now set up delay on the receiver interface:

First delete any existing queue (don't worry if there is an error, it means there was not!)

In [ ]:
for n in receiver_nodes:
    receiver_inf=n.get_interface(network_name= "link-receiver")
    receiver_inf_name = receiver_inf.get_device_name()
    n.execute("sudo tc qdisc del dev " + receiver_inf_name + " root netem")
    n.execute("sudo tc qdisc add dev " + receiver_inf_name + " root netem delay " + str(delay) +"ms limit 1000000")
    

### Start parallel servers on the receivers

In this, the base_port is the starting address of port number

Now start the `iperf3` flows:

In [ ]:
base_port=50000
for i, n in enumerate(receiver_nodes):
    n.execute("sudo killall iperf3")
    for k in range (1,num_servers+1): 
        server_port=base_port+k
        report_file=str(server_port)+"-server.dat"
        n.execute_thread("iperf3 -s -p "+str(server_port)+" -D --logfile "+report_file)

In [ ]:
#check files in all receiver
for n in receiver_nodes:
    n.execute("ls")

### Start parallel clients on the senders

In this, the base_port is the starting address of port number on the receiver


In [ ]:
import time
base_port=50000
                                  
for i, n in enumerate(sender_nodes):
    n.execute("sudo killall iperf3")
    
    for k in range (0,num_servers):
        server_port=base_port+k+1
        report_file="sender-10.10.2.1"+str(i)+"-"+str(server_port)+"-"+str(test_duration)+"-"+cca+".txt"
        n.execute_thread("iperf3 -c 10.10.2.1"+str(i)+ " -p "+str(server_port)+ " -t "+str(test_duration)+ " -C "+cca+" -P "+str(flows)+ " -i "+str(interval)+" --format k &>"+report_file+ " &")
    time.sleep(5)
    n.execute_thread(f'chmod +x cwn.sh && bash cwn.sh 10.10.2.1'+str(i))
                                       
time.sleep(test_duration+60)              

In [ ]:
#check files on all the senders
for n in sender_nodes:
    n.execute("ls")
    

In [ ]:
sender_nodes[0].execute("tail --lines=5 sender-10.10.2.10-50004-800-reno.txt")

### Get queue statistics on the router after experiment

In [ ]:
router_node.execute("tc -p -s -d -j qdisc show dev "+router_egress_name +" >tc_after.txt")

## Analyze the results

Calculate sum of bandwidth, square of sum of bandwidth, count of flows and jfi:

Calculate packet drop rate

In [ ]:
#To get packet dropped:
(drop_before,err_drop_before)=router_node.execute("tail --lines=10 tc_before.txt| grep '\"drops\":' | awk '{print $2}' |cut -d ',' -f1")

#To get packets sent
(sent_before,err_sent_before)=router_node.execute("tail --lines=10 tc_before.txt| grep '\"packets\":' | awk '{print $2}' |cut -d ',' -f1")

#To get packet dropped:
(drop_after,err_drop_after)=router_node.execute("tail --lines=10 tc_after.txt| grep '\"drops\":' | awk '{print $2}' |cut -d ',' -f1")

#To get packets sent
(sent_after, err_sent_after)=router_node.execute("tail --lines=10 tc_after.txt| grep '\"packets\":' | awk '{print $2}' |cut -d ',' -f1")

#Calculate packet drop rate:

dropped=int(drop_after)-int(drop_before)

sent=int(sent_after)-int(sent_before)
drop_rate=float(dropped)/float(sent)

print("packet drop before running experiment: "+ str(drop_before))
print("packet sent before running experiment: " + str(sent_before))
print("packet drop after running experiment: "+ str(drop_after))
print("packet sent after  running experiment: " + str(sent_after))
print("packet drop rate: " + str(drop_rate))


Run the data processing scripts on each sender to get packet loss, congestion window halving events and rtt from iperf3 and ss output.

In [ ]:
for i,n in enumerate(sender_nodes[0:1]):
    n.execute_thread(f'chmod +x process_cwn_file.py && python3 process_cwn_file.py '+str(i) )
    n.execute_thread(f'chmod +x process_iperf_normal.py && python3 process_iperf_normal.py '+str(i)+\
                                     " "+str(num_servers)+" "+str(test_duration)+" "+cca+" "+str(flows))



Run mathis_sender.py script on each sender to get packet loss rate and cwnd halving rate of each flow and save the output to packet_loss_iperf(i).csv file

In [ ]:
import time
time.sleep(test_duration+60)
for i,n in enumerate(sender_nodes):
    n.execute(f'chmod +x mathis_sender.py && python3 mathis_new_sender.py '+str(i)+\
                                      " "+str(num_servers)+" "+cca+" 10 1000 "+str(interval)+" "+str(test_duration))

Get JFI by calculating total bandwidth, sum of sq of bandwidth and count of flow.

In [ ]:

sum_bw = []
sum_sq_bw = []
count_flow = []

for n in sender_nodes:
    (sum_sen, serr)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" . |awk '{sum+=$7}END {print sum}'")
    sum_bw.append(float(sum_sen.strip()))
    (sum_sq, sqerr)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" . |awk '{sum+=$7*$7}END {print sum}'")
    sum_sq_bw.append(float(sum_sq.strip()))
    (ncount, ncerr)=n.execute("grep -r -E \"[0-9].*0.00-[0-9].*sender\" . |awk '{count+=1}END {print count}'")
    count_flow.append(int(ncount.strip()))


total_tput=sum(sum_bw)
sum_sq_tput=sum(sum_sq_bw)
count_flow=sum(count_flow)

print("Sum of bandwidth is %f Kbits/sec " % total_tput)

print("Sum of square of bandwidth is %f" % sum_sq_tput)

print("Count of flows is %d" % count_flow)

jfi= (total_tput*total_tput)/(count_flow*sum_sq_tput)

print("JFI is %f" % jfi)

Download all the packet_loss_iperf(i).csv file to the environment

In [ ]:
for i,n in enumerate(sender_nodes):
    n.download_file("/home/fabric/work/IMC6/fabric-notebooks/packet_loss_iperf"+str(i)+".csv", "/home/ubuntu/packet_loss_iperf"+str(i)+".csv")

Process the downloaded file to get the Mathis constant. Final output is saved to Mathis_C.csv file.

In [ ]:
import pandas as pd
import numpy as np
import csv
import sklearn
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import accuracy_score
import sys
import os

output_filename='Mathis_C.csv'
if not os.path.isfile(output_filename):
    with open(output_filename, 'a', newline='') as csvfile:
      writer = csv.writer(csvfile)
      header = 'time_interval', 'time_duration' ,'ports', 'sum(y_values)', 'total_cwnd_half', 'total_retransmission',\
        'total_retransmission/total_cwnd_half', 'np.nanmean(list_ratio)', 'reg_simple1.intercept_', 'reg_simple1.coef_[0]',\
        'reg_simple2.intercept_', 'reg_simple2.coef_[0]', 'router_dropped', 'router_sent', 'router_dropped/total_cwnd_half', 'mse_1', 'mse_2'
      writer.writerow(header)

df_csv_concat = pd.concat([pd.read_csv("packet_loss_iperf"+str(i)+".csv") for i in range(n_endpoints) ], ignore_index=True)
    
    
y_values = np.array(df_csv_concat['bandwidth_port'])
x1_values = np.array(x1_values).reshape(-1, 1)
x2_values = np.array(x2_values).reshape(-1, 1)


print("method-1: calculation of packet_loss rate using transfer and retrans from iperf3 data")
reg_simple1 = LinearRegression(fit_intercept = False).fit(x1_values, y_values)
print("Intercept: " , reg_simple1.intercept_)
print("Coefficient list: ", reg_simple1.coef_)

print("method-2: calculation of cwnd_halving rate using cwnd and transfer from iperf3 data")
reg_simple2 = LinearRegression(fit_intercept = False).fit(x2_values, y_values)
print("Intercept: " , reg_simple2.intercept_)
print("Coefficient list: ", reg_simple2.coef_)


y_hat1 = reg_simple1.predict(x1_values)
y_hat2 = reg_simple2.predict(x2_values)

mse1 = metrics.mean_squared_error(y_values, y_hat1)
mse2 = metrics.mean_squared_error(y_values, y_hat2)

with open(output_filename, 'a', newline='') as csvfile:
  writer = csv.writer(csvfile)
  columns = interval, test_duration, len(df_csv_concat), sum(y_values), sum(df_csv_concat['cwn_half_port']), sum(df_csv_concat['port_retrans']),\
    sum(df_csv_concat['port_retrans'])/sum(df_csv_concat['cwn_half_port']), np.nanmean(df_csv_concat['ratio']), reg_simple1.intercept_,\
    reg_simple1.coef_[0], reg_simple2.intercept_, reg_simple2.coef_[0], dropped, sent, dropped/sum(df_csv_concat['cwn_half_port']), mse1, mse2
  writer.writerow(columns)


Save the linear regression plots to a pdf.

In [ ]:
with PdfPages("linear_reg_plot.pdf") as pdf:
  plt.rcParams['figure.figsize'] = (8,6)


  plt.scatter(x=x1_values.squeeze(), y=y_values, color='C4', s=10, label='actual values')
  plt.scatter(x=x1_values.squeeze(), y=y_hat1, color='C3', s=10, label='predicted_values')
  plt.plot(x1_values.squeeze(), y_hat1, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(packet_loss_rate)") 
  plt.ylabel("y=bandwidth(bits/sec)")
  plt.title("Method-1: calculation of packet_loss rate using transfer and retrans from iperf3 data")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()

  plt.scatter(x=x2_values.squeeze(), y=y_values, color='C4', alpha=1, s=10, label='actual values')
  plt.scatter(x=x2_values.squeeze(), y=y_hat2, color='C3',  alpha=1, s=10, label='predicted_values')
  plt.plot(x2_values.squeeze(), y_hat2, color='C2', linewidth=0.5, label='fit')
  plt.xlabel("x=mss/rtt*sqrt(cwnd_half_rate)")
  plt.ylabel("y=bandwidth(bits/sec)")

  plt.title("Method-2: calculation of cwnd_halving rate using cwnd and transfer from iperf3 data")
  plt.legend()
  pdf.savefig()  # saves the current figure into a pdf page
  plt.show()
  plt.close()

Remove the transfered files after data analysis and/or downloading to your local system.

In [ ]:
%rm pac*

In [ ]:
%rm li*